<a href="https://colab.research.google.com/github/cs23m066/Assignment_2/blob/main/dl_assignment_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets

In [ ]:
import math

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-04 14:32:11--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.207, 173.194.196.207, 173.194.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   173MB/s    in 18s     

2024-04-04 14:32:29 (202 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [ ]:
!rm nature_12K.zip

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project ='DL_assignment_2')

2024-04-04 14:33:17.683766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 14:33:17.683903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 14:33:17.852087: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23m066 (dlassignment). Use `wandb login --relogin` to force relogin


In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10,numFilterss=32,sizeFilter=3,neurons=128,activFun='sigmoid',dropOut=0.0,batchNorm='no',org=0):
        super(CNN, self).__init__()
        self.activFunName = activFun
        self.batchNorm = batchNorm
        if(org ==  0):
            numFilters = [numFilterss,numFilterss,numFilterss,numFilterss,numFilterss]
        else:
            numFilters = [numFilterss,numFilterss*2,numFilterss*4,numFilterss*8,numFilterss*16]
        width = 0.0
        hight = 0.0
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=numFilters[0],kernel_size=sizeFilter, stride=1)
        width = (256 - sizeFilter)+1
        hight = (256 - sizeFilter)+1
        self.bn1 = nn.BatchNorm2d(numFilters[0])
        self.pool1 = nn.MaxPool2d(kernel_size=sizeFilter, stride=2)
        width = math.floor((width - sizeFilter)/2) + 1
        hight = math.floor((hight -sizeFilter)/2) + 1

        #print(width,hight)
        self.conv2 = nn.Conv2d( in_channels=numFilters[0], out_channels=numFilters[1], kernel_size=sizeFilter,stride=1)
        width = ((width - sizeFilter))+1
        hight = ((hight-sizeFilter))+1
        self.bn2 = nn.BatchNorm2d(numFilters[1])
        self.pool2 = nn.MaxPool2d(kernel_size=sizeFilter,stride=2)
        width = math.floor((width - sizeFilter)/2) + 1
        hight = math.floor((hight -sizeFilter)/2) + 1

        #print(width,hight)
        self.conv3 = nn.Conv2d( in_channels=numFilters[1], out_channels=numFilters[2], kernel_size=sizeFilter,stride=1)
        width = ((width - sizeFilter))+1
        hight = ((hight-sizeFilter))+1
        self.bn3 = nn.BatchNorm2d(numFilters[2])
        self.pool3 = nn.MaxPool2d(kernel_size=sizeFilter,stride=2)
        width = math.floor((width - sizeFilter)/2) + 1
        hight = math.floor((hight -sizeFilter)/2) + 1

        #print(width,hight)
        self.conv4 = nn.Conv2d( in_channels=numFilters[2], out_channels=numFilters[3], kernel_size=sizeFilter,stride=1)
        width = ((width - sizeFilter))+1
        hight = ((hight-sizeFilter))+1
        self.bn4 = nn.BatchNorm2d(numFilters[3])
        self.pool4 = nn.MaxPool2d(kernel_size=sizeFilter,stride=2)
        width = math.floor((width - sizeFilter)/2)+ 1
        hight = math.floor((hight -sizeFilter)/2) + 1

        #print(width,hight)
        self.conv5 = nn.Conv2d( in_channels=numFilters[3], out_channels=numFilters[4], kernel_size=sizeFilter,stride=1)
        width = ((width - sizeFilter))+1
        hight = ((hight-sizeFilter))+1
        self.bn5 = nn.BatchNorm2d(numFilters[4])
        self.pool5 = nn.MaxPool2d(kernel_size=sizeFilter,stride=2)
        width = math.floor((width - sizeFilter)/2) + 1
        hight = math.floor((hight -sizeFilter)/2) + 1

        #print(width,hight)
        self.dropout = nn.Dropout(p=dropOut)
        self.fc1 = nn.Linear(numFilters[4] * width*hight, neurons)
        self.bn6 = nn.BatchNorm1d(neurons)
        self.fc2 = nn.Linear(neurons,10)

    def forward(self, x):
        if(self.activFunName == 'relu'):
            activation_fn = F.relu
        elif(self.activFunName == 'gelu'):
            activation_fn = F.gelu
        elif(self.activFunName == 'silu'):
            activation_fn = F.silu
        else:
            activation_fn = F.mish

        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn1(self.conv1(x)))
        else:
            x = activation_fn(self.conv1(x))
        x = self.pool1(x)

        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn2(self.conv2(x)))
        else:
            x = activation_fn(self.conv2(x))
        x = self.pool2(x)

        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn3(self.conv3(x)))
        else:
            x = activation_fn(self.conv3(x))
        x = self.pool3(x)

        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn4(self.conv4(x)))
        else:
            x = activation_fn(self.conv4(x))
        x = self.pool4(x)

        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn5(self.conv5(x)))
        else:
            x = activation_fn(self.conv5(x))
        x = self.pool5(x)

        x = x.reshape(x.shape[0], -1)
        if(self.batchNorm == 'yes'):
            x = activation_fn(self.bn6(self.fc1(x)))
        else:
            x = activation_fn(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])

transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by a maximum of 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(256),  # Randomly crop and resize the image to 256x256
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5,),(0.5,))  # Normalize the image
])

train_dataset2 = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform2)
train_dataset_aug,val_dataset_aug = torch.utils.data.random_split(train_dataset2,[8000,1999])

def dataFun(aug,batchSize):
    if(aug == 'no'):
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        val_loader = torch.utils.data.DataLoader(val_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        return train_loader,val_loader
    else:
        train_loader_aug = torch.utils.data.DataLoader(train_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        val_loader_aug = torch.utils.data.DataLoader(val_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        return train_loader_aug,val_loader_aug

In [ ]:
def train_fun(neurons,numFilters,sizeFilter,activFun,optimizerName,batchSize,dropOut,num_epochs,learning_rate,batchNorm,aug,org):

    train_loader,val_loader = dataFun(aug,batchSize)

    in_channels = 3
    num_classes = 10

    model = CNN(in_channels, num_classes,numFilters,sizeFilter,neurons,activFun,dropOut,batchNorm,org).to(device)

    if(optimizerName == 'sgd'):
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    elif(optimizerName == 'adam'):
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    else:
        optimizer = optim.NAdam(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)
            # forward
            scores = model(data)
            loss = criterion(scores,targets)
            # backward
            optimizer.zero_grad()
            loss.backward()
            # gradient descent or adam step
            optimizer.step() #updates the parameters
        train_accuracy,train_loss = check_accuracy(train_loader, model,criterion,batchSize)
        validation_accuracy,validation_loss = check_accuracy(val_loader, model,criterion,batchSize)
        print(f"train_accuracy:{train_accuracy:.4f},train_loss:{train_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})

    #wandb.log({'train_accuracy':train_accuracy})


In [ ]:
def check_accuracy(loader,model,criterion,batchSize):
    num_correct = 0
    num_loss = 0
    total = 0
    num_samples = 0
    total_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batchSize
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples)*100 , total_loss

In [ ]:
# neurons = 128
# numFilters = 32
# sizeFilter = 3
# activFun = 'relu'
# optimizer = 'adam'
# batchSize=32
# dropOut = 0
# num_epochs = 5
# learning_rate = 1e-3
# batchNorm = 'yes'
# aug='yes'
# org = 1
# train_fun(neurons,numFilters,sizeFilter,activFun,optimizer,batchSize,dropOut,num_epochs,learning_rate,batchNorm,aug,org)

In [ ]:
def main_fun():
    wandb.init(project ='DL_assignment_2')
    params = wandb.config
    with wandb.init(project = 'DL_assignment_2', name='neurons'+str(params.neurons)+'optimizer'+params.optimizer+'activFun'+params.activFun) as run:
        train_fun(params.neurons,params.numFilters,params.sizeFilter, params.activFun,params.optimizer,params.batchSize,params.dropOut,params.num_epochs,params.learning_rate,params.batchNorm,params.aug,params.org)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'Shriram',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'neurons':{'values':[128,256]},
        'numFilters':{'values':[32,64]},
        'sizeFilter':{'values' : [3,5]},
        'activFun' :{'values':['relu','gelu','silu','mish']},
        'optimizer' :{'values':['adam','nadam']},
        'batchSize' : {'values':[32,64]},
        'dropOut' :{'values':[0,0.2,0.4]},
        'num_epochs':{'values':[5,10]},
        'learning_rate' :{'values':[1e-3,1e-4]},
        'batchNorm' :{'values' : ['yes','no']},
        'aug' : {'values':['no']},
        'org' : {'values':[0,1]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'DL_assignment_2')
wandb.agent(sweepId,function =main_fun,count = 10)
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: w3vbxs5d
Sweep URL: https://wandb.ai/dlassignment/DL_assignment_2/sweeps/w3vbxs5d


wandb: Agent Starting Run: ekkhzpuq with config:
wandb: 	activFun: relu
wandb: 	aug: no
wandb: 	batchNorm: yes
wandb: 	batchSize: 32
wandb: 	dropOut: 0.2
wandb: 	learning_rate: 0.001
wandb: 	neurons: 128
wandb: 	numFilters: 64
wandb: 	num_epochs: 5
wandb: 	optimizer: nadam
wandb: 	org: 0
wandb: 	sizeFilter: 5


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

100%|██████████| 125/125 [00:51<00:00,  2.44it/s]


train_accuracy:60.0750,train_loss:9881.0102
validation_accuracy:43.4217,validation_loss:3465.2262


train_accuracy,▁▂▃▄▅▄▅▅▇█
train_loss,█▇▆▅▅▄▃▄▂▁
val_accuracy,▁▁▃▅▆▅▅▅▅█
val_loss,█▇▅▃▃▃▂▄▃▁
train_accuracy,60.075
train_loss,9881.01022
val_accuracy,43.42171
val_loss,3465.22615


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7b8db72c7820>> (for post_run_cell), with arguments args (<ExecutionResult object at 7b8d42122cb0, execution_count=12 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7b8d42122ef0, raw_cell="def main_fun():
    wandb.init(project ='DL_assign.." store_history=True silent=False shell_futures=True cell_id=ceede340-b1ed-4af5-9485-4497bff10ef7> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe